In [2]:
''' ONE-VERSUS-REST

Com uma abordagem mais simples onde um classificador binário é treinado para cada classe'''


import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# Passo 1: Treinamento do Modelo
# Carregar conjunto de dados IRIS
iris = load_iris()
X = iris.data
y = iris.target

# Dar nome aos bois ( so pra deixar mais claro )
class_names = iris.target_names
feature_names = iris.feature_names


# Treinar modelo de regressão logística
model = LogisticRegression(multi_class='ovr')  # Usando a estratégia "um versus o resto"
model.fit(X, y)

# Passo 2: Explicação Mínima
def minimal_explanation(model, instance):
    # Obtendo as probabilidades previstas para cada classe
    probs = model.predict_proba(instance.reshape(1, -1)).flatten()
    # Ordenar as probabilidades e índices das classes em ordem crescente
    sorted_indices = np.argsort(probs)
    sorted_probs = probs[sorted_indices]
    # Extraindo as classes com a maior probabilidade
    top_classes = sorted_indices[-3:]  # Pegar as três maiores probabilidades
    # Calculando as contribuições das features para as classes principais
    contributions = []
    for cls in top_classes:
        class_probs = np.zeros_like(probs) # Criando um array de zeros 
        class_probs[cls] = sorted_probs[-1] # garantindo que a probabilidade da classe sja diferente de zero
        contributions.append((model.coef_[cls] * class_probs[cls]).flatten()) # Multiplicando o modelo pela probab. da classe para obter as contribuições das features e salvando na lista contributions.
    return np.array(contributions) # Retornando as contribuições como array com numpy

# Passo 3: Teste com IRIS Dataset
# Selecionar uma instância de teste do conjunto de dados IRIS
instance_index = 0
instance = X[instance_index]

# Obter explicação mínima para a instância selecionada
explanation = minimal_explanation(model, instance)

# Prever a classe para a instância de teste
predicted_class = model.predict(instance.reshape(1, -1))

# Exibir a explicação mínima
print("Explicação Mínima para a Instância de Teste:")
print(f"Classe Prevista: {class_names[predicted_class[0]]}")
for i, cls in enumerate(explanation):
    print(f"Para a classe {class_names[i]}:")
    for j, feature in enumerate(cls):
        print(f"{feature_names[j]}: {feature}")


Explicação Mínima para a Instância de Teste:
Classe Prevista: setosa
Para a classe setosa:
sepal length (cm): -0.35218506849831155
sepal width (cm): -0.46134731198410894
petal length (cm): 2.628237951720523
petal width (cm): 2.1667802503737237
Para a classe versicolor:
sepal length (cm): -0.16068686159326173
sepal width (cm): -1.9091229122539757
petal length (cm): 0.6248264270869114
petal width (cm): -1.1434417953080562
Para a classe virginica:
sepal length (cm): -0.3996653809248001
sepal width (cm): 0.8076216014439159
petal length (cm): -2.080816560777914
petal width (cm): -0.8798894566148193


com base na explicação:
Classe Setosa:
    características mais influentes foram Comp. da Pétala com 2.62823...
                                          Larg. da Pétala com 2.16678...
    as outras características foram negativas com bem menos influência

Classe Versicolor:
    características mais influentes foram Comp. da Pétala com 0.62482...
    as outras foram negativas

Classe Virínica:
    característica mais influentes foram Largura da Sépala com 0.80762...
    as outras foram negativas

In [3]:
''' SOFTMAX
O softmax é mais indicado para problemas de classificação multiclasse pois as previsões são expressas como probabilidades para cada classe '''

import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# Passo 1: Treinamento do Modelo
# Carregar conjunto de dados IRIS
iris = load_iris()
X = iris.data
y = iris.target

# Treinar modelo de regressão logística com a estratégia softmax
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)

# Passo 2: Explicação Mínima
def minimal_explanation(model, instance):
    # Obtendo as probabilidades previstas para cada classe
    probs = model.predict_proba(instance.reshape(1, -1)).flatten()
    # Ordenar as probabilidades e índices das classes em ordem crescente
    sorted_indices = np.argsort(probs)
    sorted_probs = probs[sorted_indices]
    # Extraindo as classes com a maior probabilidade
    top_classes = sorted_indices[-3:]  # Pegar as três maiores probabilidades
    # Calculando as contribuições das features para as classes principais
    contributions = []
    for cls in top_classes:
        class_probs = np.zeros_like(probs)
        class_probs[cls] = sorted_probs[-1]
        contributions.append((model.coef_[cls] * class_probs[cls]).flatten())
    return np.array(contributions)

# Passo 3: Teste com IRIS Dataset
# Selecionar uma instância de teste do conjunto de dados IRIS
instance_index = 0
instance = X[instance_index]

# Obter explicação mínima para a instância selecionada
explanation = minimal_explanation(model, instance)

# Prever a classe para a instância de teste
predicted_class = model.predict(instance.reshape(1, -1))

# Exibir a explicação mínima
print("Explicação Mínima para a Instância de Teste:")
print(f"Classe Prevista: {class_names[predicted_class[0]]}")
for i, cls in enumerate(explanation):
    print(f"Para a classe {i}:")
    for j, feature in enumerate(cls):
        print(f"Feature {j+1}: {feature}")


Explicação Mínima para a Instância de Teste:
Classe Prevista: setosa
Para a classe 0:
Feature 1: -0.10594283498933536
Feature 2: -0.6384008398776853
Feature 3: 2.6719993174241807
Feature 4: 1.9923363818652748
Para a classe 1:
Feature 1: 0.5197923251415844
Feature 2: -0.30886032735733576
Feature 3: -0.19793195878541697
Feature 4: -0.927740553607467
Para a classe 2:
Feature 1: -0.4138494901522469
Feature 2: 0.9472611672350082
Feature 3: -2.4740673586387945
Feature 4: -1.064595828257813


c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
